# Chapter 3: Attention Mechanisms

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.9.0


`inputs`:  $x^{(i)}_{a}$

$i$ is the sequence index and $a$ is the feature index

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

`query`: $q_{a} = x^{(2)}_{a}$

`attn_scores_2`: $\omega_{2i} = \sum_{a} q_{a} x^{(i)}_{a}$

In [3]:
query = inputs[1] #x(2) in book
attn_scores_2 = torch.empty(inputs.shape[0])
for i,x in enumerate(inputs):
    attn_scores_2[i] = torch.dot(inputs[i],query)
print('un-normalized scores:',attn_scores_2)

attn_scores_2_tmp = attn_scores_2 / attn_scores_2.sum()
print('normalized scores:', attn_scores_2_tmp)

un-normalized scores: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
normalized scores: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


Softmax implementation
Input is `x` which is expected to be the attention scores $\omega_{2i}$, here is we are using element 2 as the query, this is a 1-D vector in the sequence index.

Attention weights `attn_weights_2` : $\alpha_{2i} = \text{softmax}(\omega_{2i})$

In [4]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum()
attn_weights_2= softmax_naive(attn_scores_2)
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [5]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


`context_vec_2`: $z^{(2)}_{a} = \sum_i \alpha_{2i} x^{(i)}_{a}$

In [6]:
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Generalizing to all input sequence tokens
`attn_scores` = $\omega_{ij} = \sum_{a} x^{(i)}_{a}x^{(j)}_{a}$, which is an $n \times n$ tensor, where $n$ is the sequence index.

In [7]:
n_seq = inputs.shape[0]
attn_scores = torch.empty(n_seq,n_seq)

for i,x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i][j] = torch.dot(x_i,x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [8]:
attn_scores = inputs @ inputs.T # 6x3 . (6x3)^T = 6x3 . 3x6
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


`attn_weights` = $\alpha_{ij} = \text{softmax}_{j}(\omega_{ij})$ 

In [9]:
attn_weights = torch.softmax(attn_scores,dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [10]:
## check that the rows sum to 1
attn_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

`all_context_vecs` = $z^{i}_{a} = \sum_{j} x^{j}_{a} \alpha_{ij}$

(n,d) = (n,n) x (n,d)

In [11]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [12]:
print(all_context_vecs[1]) #should match what we had before
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])
tensor([0.4419, 0.6515, 0.5683])


## 3.4 Trainable weights

In this example the model dimension is `d_in` (3) but the Q,K,V vectors live in a
different, lower dimensional space `d_out` (2). Presumably this makes it a little easier to generalize to multiple heads where `d_out` typically is `d_in / n_heads`

In [13]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
torch.manual_seed(123)
W_q = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

q_2 = x_2 @ W_q # (3) x (3x2) -> 2
k_2 = x_2 @ W_k
v_2 = x_2 @ W_v

print(q_2)

tensor([0.4306, 1.4551])


In [14]:
keys = inputs @ W_k
values = inputs @ W_v
queries = inputs @ W_v

In [15]:
#omega_{22}
keys_2 = keys[1]
attn_score_22 = q_2.dot(k_2)
print(attn_score_22)

tensor(1.8524)


`attn_scores_2` = $\omega_{2j} = \sum_{a} q^{(2)}_{a} k^{(j)}_{a}$

In [16]:
attn_scores_2 = q_2 @ keys.T
print('omega_{2j}: ',attn_scores_2)

omega_{2j}:  tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


`attn_weights_2` = $\alpha_{2j} = \text{softmax}(\omega_{2j}/\sqrt{d_{out}},j)$

In [17]:
attn_weights_2 = torch.softmax(attn_scores_2/d_out**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


`context_vec_2` = $z^{(2)}_{a} = \sum_{j}  \alpha_{2j} v^{(j)}_{a}$

In [18]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


#### Self attention class
`class SelfAttention_v1`

Subclass `nn.Module`. Implement `__init__`, and `forward(self,x)`

In [19]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        #define the weights
        self.W_q = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_k = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_v = torch.nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x): # x: shape(...,n,d_in)

        #shape(...,n,d_in) x (d_in,d_out) -> shape(...,n,d_out)
        queries = x @ self.W_q 
        keys = x @ self.W_k
        values = x @ self.W_v
        

        #(n,d_out) x (n,d_out)^T -> shape(n,n)
        omega = queries @ keys.T
        alpha = torch.softmax(omega/d_out**0.5,dim=-1)
        context_vec = alpha @ values
        return context_vec

In [20]:
#now test it
torch.manual_seed(123)
SA_v1 = SelfAttention_v1(d_in,d_out)
SA_v1.forward(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

#### New self attention class using linear layers for holding parameters
`class SelfAttention_v2`

Subclass `nn.Module`. Implement `__init__`, and `forward(self,x)`

In [21]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        #define the weights
        self.W_q = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out,bias=qkv_bias)

    def forward(self, x): # x: shape(...,n,d_in)

        #shape(...,n,d_in) x (d_in,d_out) -> shape(...,n,d_out)
        queries = self.W_q(x) 
        keys = self.W_k(x)
        values = self.W_v(x)
        

        #(n,d_out) x (n,d_out)^T -> shape(n,n)
        omega = queries @ keys.T
        alpha = torch.softmax(omega/d_out**0.5,dim=-1)
        context_vec = alpha @ values
        return context_vec
    
torch.manual_seed(789)
SA_v2 = SelfAttention_v2(d_in,d_out)
SA_v2.forward(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

## 3.5 Causal attention and masking

In [22]:
#grabbing some of the attention weights to demonstrate masking
queries = SA_v2.W_q(inputs)
keys = SA_v2.W_k(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [23]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [24]:
masked_simple = attn_weights*mask_simple
print(masked_simple) #same as above but with zeros above diagonal

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [25]:
#this wrecks the normalization
row_sums = masked_simple.sum(dim=-1,keepdim = True) #sums over columns for each row
masked_simple_norm = masked_simple / row_sums # M_ij = A_ij / sum_k A_ik
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


##### Note on `keepdim`

If this is `keepdim = True` then `row_sums.shape` is (6,1)

`masked_simple_norm = masked_simple / row_sums` broadcasts the denominator so it is (6,6) where each of the 6 columns is the same as the first so
$$M_{ij} = A_{ij} / \sum_{k} A_{ik}$$
That is, the denominator does not depend on the column index $j$, it is the same for all columns meaning that when a given row of $M$ is summed over the columns the result is 1. `masked_simple_norm.sum(dim=-1)`:
$$ \sum_j M_{ij} = \sum_{j} A_{ij} / \sum_{k} A_{ik} = 1\quad\forall\quad i$$


If this is `keepdim =False` then `row_sums.shape` is (6,). The effect of broadcasting is different. The rules for broadcasting the division operation will align the first index of this tensor with the last index of the numerator.
$$(6,6) / (6,) \rightarrow (6,6) / (1,6) \rightarrow (6,6) / (6,6)$$

In this case the tensor in the denominator $\sum_{k} A_{lk}$ the free index is matched to the index $j$ in the numerator:
$$M_{ij} = A_{ij} / \sum_{k} A_{jk}$$

The denominator is now independent of the _row index_ $i$. Thus for each row the elements are divided by different column-dependent factors, but the division is the same for each row. 
For the normalization  `masked_simple_norm.sum(dim=-1)`:

$$ \sum_j M_{ij} = \sum_{j} A_{ij} / \sum_{k} A_{jk} \neq 1\quad\forall\quad i$$

You *can't* fix this with `masked_simple_norm.sum(dim=0)` because the this version mixes up rows and columns via broadcasting.


#### Now do this avoiding the need for a final normalization by masking the attention _scores_ ($\omega$) instead of the _weights_ ($\alpha$), by noting that a weight of zero can be obtained by assigning the scores to $-\infty$ 
$$ \alpha_{ij} = 0 \leftrightarrow \omega_{ij} = -\infty$$


In [26]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)
masked = attn_scores.masked_fill(mask.bool(),-torch.inf)
print(masked)
attn_weights = torch.softmax(masked/keys.shape[-1]**0.5,dim=-1)
print('new way\n')
print(attn_weights)
print('old way\n')
print(masked_simple_norm)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)
new way

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
old way

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.23

#### Adding dropout

In [27]:
#first explore pytorch dropout layer functionality
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) #thats 50%
example = torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [28]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


#### 3.5.3 Implementing causal attention including functionality for extra batch dimension

In [29]:
#create a tensor with batch structure
batch = torch.stack((inputs,inputs),dim=0) 
# we stacked two copies of input using index 0
# thus batch.shape[0] should be 2
print(batch.shape)

torch.Size([2, 6, 3])


In [30]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
         #define the weights
        self.W_q = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.score_mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)
    
    #now x has shape (batch, seq, feature)
    def forward(self, x):
        query = self.W_q(x)
        key = self.W_k(x)
        value = self.W_v(x)

        # want (batch,nseq, nseq) <- (batch,nseq,d_out) x (batch,d_out,nseq)
        #need transpose to swap last two rows 
        omega = query @ key.transpose(-2,-1)
        masked = omega.masked_fill(mask.bool(),-torch.inf)
        alpha = self.dropout(torch.softmax(masked/d_out**0.5,dim=-1))
        
        #in value (batch,nseqj,d_out), alpha (batch,nseqi,nseqj)
        # out (batch,nseq,d_out) 
        # keeping the order value.trasponse(-2,-1) @ alpha.transpose(-2,-1)
        #does the right multiplication, but needs a final transpose(-2,-1) on the
        #output to move d_out to the back.
        #distributing this transpose over the matrix product just gives
        context_vector = alpha @ value
        return context_vector

In [31]:
torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


## 3.6 Multiheaded attention!!

In [32]:
#first a loop version
#wrapper class

class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
           [CausalAttention(d_in,d_out,context_length,dropout,qkv_bias)
           for _ in range(num_heads)]
        )

    def forward(self,x):
        return torch.cat([h(x) for h in self.heads],dim = -1)
    
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out, context_length, nheads = 3, 2, batch.shape[1], 2
mha = MultiHeadAttentionWrapper(d_in,d_out,context_length,0.,nheads)
context_vecs = mha(batch)
print(context_vecs)
print("output shape: ",context_vecs.shape)
        

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
output shape:  torch.Size([2, 6, 4])


In [35]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, \
            'd_out must be divisible by num_heads'

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_q = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.register_buffer('mask', torch.triu(torch.ones(context_length,context_length),diagonal=1))
        self.dropout = nn.Dropout(dropout)
        self.out_proj = nn.Linear(d_out, d_out)


    def forward(self,x):
        b, nseq, d_in = x.shape
        query = self.W_q(x)
        key = self.W_k(x)
        value = self.W_v(x)

        #now split along n_heads
        query = query.view(b,nseq,self.num_heads,self.head_dim)
        key = key.view(b,nseq,self.num_heads,self.head_dim)
        value = value.view(b,nseq,self.num_heads,self.head_dim)

        #move n_head dimension in front of n_seq dim
        query = query.transpose(1,2)
        key = key.transpose(1,2)
        value = value.transpose(1,2)

        #next is dot product attention scores, matmul over the head_dim dimension
        # (b,n_head,n_seq,d_head) x (b,n_head,d_head,n_seq)
        # output is (b,n_head,n_seq,n_seq')
        omega = query @ key.transpose(-2,-1)

        mask = self.mask.bool()[:nseq, :nseq]
        omega = omega.masked_fill(mask, -torch.inf)
        alpha = torch.softmax(omega/self.head_dim**0.5,dim = -1)
        alpha = self.dropout(alpha)
        # shapes are 
        # alpha ~ (b,n_head,n_seq,n_seq')
        # value ~ (b,n_head,n_seq,d_head) 
        #context_vec ~ (b,n_head,d_seq,d_head)
        context_vec = alpha @ value
        #now put n_head next to d_head to roll back up
        context_vec = context_vec.transpose(1,2)
        context_vec = context_vec.contiguous().view(b,nseq,self.d_out)

        #now it is (b,n_seq,d_out), same shape as x except d_in -> d_out
        context_vec = self.out_proj(context_vec)
        return context_vec



batch_size, context_length, d_in = batch.shape
d_out = 2
torch.manual_seed(123)
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
